# Notebook 3: quantum Fourier transform and applications

In this notebook we will implement the quantum Fourier transform and explore some of its applications. 

## Quantum Fourier transform

Recall that the quantum Fourier transform is given by the following formula (eq. (5.4) in Nielsen-Chuang):
$$
    |j_1,\dots, j_n\rangle \mapsto 2^{-n/2}(|0\rangle + \exp(2\pi i0.j_n)|1\rangle) (|0\rangle + \exp(2\pi i0.j_{n-1}j_n)|1\rangle)\cdots (|0\rangle + \exp(2\pi i0.j_1j_2\dots j_n)|1\rangle)
$$

where $0.k_1k_2k_3$ denotes a number in binary form, i.e. it's $0+k_12^{-1} + k_2 2^{-2}+k_3 2^{-3}$.

The quantum Fourier transform can be implemented by a relatively simple circuit, which we will build piece by piece. It only uses two different kind of gates, the Hadamard $H$ gate and the controlled-$R_k$ gate, where 
$$
    R_k = \begin{pmatrix}
        1 & 0 \\
        0 & e^{2\pi i/2^k}
    \end{pmatrix}
$$

Note that $R_k$ can be expressed as a matrix power of the $Z$ gate. Since 
$$
Z = 
\begin{pmatrix}
1 & 0 \\
0 & -1
\end{pmatrix}
    = 
\begin{pmatrix}
1 & 0 \\
0 & e^{\pi i}
\end{pmatrix}
$$

We can easily see that $R_k = Z^{-2^{-k+1}}$. We can implement this using the controlled Z power gate, `cirq.CZPowGate` ([see documentation](https://quantumai.google/reference/python/cirq/ops/ZPowGate))

In [ ]:
import numpy as np

# Import cirq, install it if it's not installed.
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    print("installed cirq.")
    import cirq

### Exercise 1a
> Implement the main piece of the quantum Fourier transform in a function. This function takes as argument a `circq.Circuit` and a list of qubits. It applies the Hadamard transform to the first qubit, then applies a controlled $R_2$ gate to the first and second qubit, a controlled $R_3$ gate to the first and third qubit, and so on. All the operations are appended to the circuit. 

For 4 qubits the resulting circuit should look like this:

```
0: ───H───@────────@─────────@──────────
          │        │         │
1: ───────@^-0.5───┼─────────┼──────────
                   │         │
2: ────────────────@^-0.25───┼──────────
                             │
3: ──────────────────────────@^(-1/8)───
```

In [ ]:
def apply_qft_piece(circuit, qubits):
    """
    Apply the Hadamard gate to the first qubit, followed by a sequence of
    controlled R_k gates conditional on the other qubits.
    """

    first_qubit = qubits[0]
    other_qubits = qubits[1:]

    ## YOUR CODE HERE

    # Apply hadamard gate to first qubit

    # Apply `CZPowGate` to all qubits in a for loop


circuit = cirq.Circuit()
num_qubits = 4
qubits = cirq.LineQubit.range(num_qubits)  # Easy way to get a list of qubits.

apply_qft_piece(circuit, qubits)
circuit


### Exercise 1b

> Use the function you just implemented to implement the quantum Fourier transform. We first use `apply_qft_piece` to the entire list of qubits, then we apply the same function to all but the first qubit. This repeats until we only apply `apply_qft_piece` to the last qubit. (See figure 5.1 in Nielsen-Chuang). This is finally followed by a permutation reversing the order of all the qubits using `cirq.QubitPermuationGate`.

_Hint: `cirq.QubitPermuationGate` works on a sequence of arguments and not directly on a list. For example, we can do `G = cirq.QubitPermutationGate((0,2,1))` to swap qubits 1 and 2, leaving 0 untouched. To apply it to 3 qubits we should do `G(qubit0,qubit1,qubit2)`, and not `G([qubit0,qubit1,qubit2])`. This is a problem if we just have a list `qubits` of arbitrary length. Fortunately, Python allows 'unpacking' a list using `*`, which will unpack the list into individual arguments. That is, we can call `G(*qubits)`, or `G(*[qubit0,qubit1,qubit2])`._

When applied to 4 qubits, the circuit should look like this:

```
                   ┌────────┐   ┌──────────────┐   ┌────────┐
0: ───H───@─────────@────────────@───────────────────────────────────────────[0>3]───
          │         │            │                                           │
1: ───────@^-0.5────┼──────H─────┼───────@──────────@────────────────────────[1>2]───
                    │            │       │          │                        │
2: ─────────────────@^-0.25──────┼───────@^-0.5─────┼──────H────@────────────[2>1]───
                                 │                  │           │            │
3: ──────────────────────────────@^(-1/8)───────────@^-0.25─────@^-0.5───H───[3>0]───
                   └────────┘   └──────────────┘   └────────┘
```

In [ ]:
def apply_qft(circuit, qubits):
    num_qubits = len(qubits)
    # YOUR CODE HERE

    # Use `apply_qft_piece` in a for loop

    # Apply the `QubitPermutationGate`


circuit = cirq.Circuit()
num_qubits = 4
qubits = cirq.LineQubit.range(num_qubits)

apply_qft(circuit, qubits)
circuit


The quantum Fourier transform essentially acts by applying the discrete Fourier transform to the coefficients of the qubits. We can use this fact to verify that the circuit you implemented above is correct. The difference between the two methods should be on the order of single precision machine epsilon (~1e-8). If it is bigger, you probably did something wrong, and you should check your implementation carefully.

In [ ]:
# Create quantum circuit and simulator
num_qubits = 4
circuit = cirq.Circuit()
qubits = cirq.LineQubit.range(num_qubits)
apply_qft(circuit, qubits)
simulator = cirq.Simulator()

# Use random norm-1 vector
init_state = np.random.normal(size=2**num_qubits)
init_state /= np.linalg.norm(init_state)

# Quantum version
result = simulator.simulate(circuit, initial_state=init_state)
quantum = result.state_vector()

# Classical version
# norm='ortho' to make FFT unitary
classical = np.fft.fft(init_state, norm="ortho")

diff = quantum - classical
error = np.linalg.norm(diff)
print("Difference between classical DFT and your circuit:", error)

if error > 1e-6:
    print("Your circuit is not working correctly. Here's the difference:")
    print(diff)

    print("Here are the quantum and classical results respectively:")
    print(quantum)
    print(classical)
else:
    print("Looks like your circuit works great!")


In fact, we can also just convert the circuit to a unitary matrix, and compare this to the unitary matrix of the classical discrete fourier transform. The difference between these two should also be small.

In [ ]:
np.linalg.norm(
    circuit.unitary() - np.fft.fft(np.eye(2**num_qubits), norm="ortho")
)

## Phase estimation

The phase estimation quantum algorithm can be used to find the eigenvalue $\exp(2\pi i \varphi)$ associated to an eigenvector $|u\rangle$ of a unitary operator $U$. Here $U$ is treated like a black box, and we assume we have some way to prepare the state $|u\rangle$, without knowing the phase $\varphi$.

This may seem like a strange requirement, but as we have seen it is an essential ingredient in certain algorithms, like order finding / integer factoring (and hence Shor's algorithm which can break RSA cryptography in theory). 


We start by generating a random unitary matrix $U$ and record one of its eigenvectors and eigenvalues. 
_If you're interested in random matrices, the code below generates unitary matrices uniformly random with respect to the Haar measure on_ $U(2^n)$.

In [ ]:
def random_unitary(input_size, seed=None):
    """Create a random unitary matrix of size `2**input_size`"""
    if seed is not None:
        np.random.seed(seed)
    U = np.random.normal(
        size=(2**input_size, 2**input_size)
    ) + 1j * np.random.normal(size=(2**input_size, 2**input_size))
    U, R = np.linalg.qr(U / np.sqrt(2))
    U = U @ np.diag(np.sign(np.diag(R)))
    return U


input_size = 3
U = random_unitary(input_size)
eigvals, eigvects = np.linalg.eig(U)
u = eigvects[0]
phi = np.angle(eigvals[0]) / (2*np.pi)
print("phi:", phi)
print("u:", u)


Now the phase estimation algorithm works with two _registers_. The first register are _working qubits_, and we'll manipulate them based on the input to the second register. The second register will contain the eigenvector $u$. 

The first step is to apply a Hadamard gate to all the qubits in the first register. The second step is to a controlled $U^{2^j}$ gate to each of the qubits in the first register. Then finally we apply an inverse quantum Fourier transform to the qubits in the first register. Please look at figure 5.2 in Nielsen-Chuang for a visual explanation of the algorithm.

Rather than plugging in an initial state when simulating the circuit, it will be easier to just use the standard state `|0...0>` as input. However, this means that we need to _prepare_ the second register to the state `|u>`. Fortunately, in this case $u$ is the first eigenvector of the matrix $U$. If $V$ is the matrix of eigenvectors of $U$, then $|u\rangle = V|0\rangle$. Since $V$ is unitary, we can use it as a quantum gate to prepare the second register.

### Exericse 2
> Implement the phase estimation algorithm. As input you should take the unitary matrix $U$, a list of qubits of the first register, and a list of qubits of the second register. The function should add all the required operations to the circuit. Pay attention which qubit is the control qubit for the controlled $U^{2^j}$ operations. You might might need the same 'list unpacking' trick as in exercise 1b.

Rather than implementing the inverse Fourier transform like in Exercise 1, we'll cheat an directly implement it based on its matrix representation. This only works for a small amount of qubits; otherwise this would be too expensive to do. 


The final result should look like this:
```
0: ───H──────────────────────────────────────────────────────────────────@──────────────ift[1]───M───
                                                                         │              │
1: ───H───────────────────────────────────────────────────@──────────────┼──────────────ift[2]───M───
                                                          │              │              │
2: ───H────────────────────────────────────@──────────────┼──────────────┼──────────────ift[3]───M───
                                           │              │              │              │
3: ───H─────────────────────@──────────────┼──────────────┼──────────────┼──────────────ift[4]───M───
                            │              │              │              │              │
4: ───H──────@──────────────┼──────────────┼──────────────┼──────────────┼──────────────ift[5]───M───
             │              │              │              │              │
5: ───u[1]───U**(2**0)[1]───U**(2**1)[1]───U**(2**2)[1]───U**(2**3)[1]───U**(2**4)[1]────────────────
      │      │              │              │              │              │
6: ───u[2]───U**(2**0)[2]───U**(2**1)[2]───U**(2**2)[2]───U**(2**3)[2]───U**(2**4)[2]────────────────
      │      │              │              │              │              │
7: ───u[3]───U**(2**0)[3]───U**(2**1)[3]───U**(2**2)[3]───U**(2**3)[3]───U**(2**4)[3]────────────────
```

In [ ]:
register_size = 5

# Gate representing the inverse Fourier transform.
ift_gate = cirq.MatrixGate(
    (np.fft.fft(np.eye(2**register_size), norm="ortho")), name="ift"
)

# Gate representing eigenvectors of U
prep_gate = cirq.MatrixGate(eigvects, name="u")


def CU_pow_gate(power):
    """Create a controlled U**(2**power) gate."""
    U_pow = np.linalg.matrix_power(U, 2**power)
    U_pow_gate = cirq.MatrixGate(U_pow, name=f"U**(2**{power})")
    CU_gate = cirq.ControlledGate(U_pow_gate, num_controls=1)
    return CU_gate


def phase_estimation(circuit, register1, register2):
    # YOUR CODE HERE

    # Apply hadamard gates to first register

    # Prepare second register to state |u>

    # Apply the conditional U**(2**j) gates

    # Apply the inverse Fourier transform to the first register

    # Measure first register

    ...


register1 = cirq.LineQubit.range(register_size)
register2 = cirq.LineQubit.range(register_size, register_size + input_size)
phase_estimation_circuit = cirq.Circuit()

phase_estimation(phase_estimation_circuit, register1, register2)
phase_estimation_circuit


Now we can do a simulation to check the circuit works. We should get an estimated phase close to the target phase $\varphi$. 

To get a better approximation, all we need to do is to increase the size of the first register.

In [ ]:
simulator = cirq.Simulator()
result = simulator.run(phase_estimation_circuit)
result = np.array(result.data).reshape(-1)
estimated_phase = np.dot(2.0**(-np.arange(register_size)-1), result)

print("True phase:",np.mod(phi,1))
print("Estimated phase:", estimated_phase)